In [2]:
%load_ext autoreload
%autoreload 2

import pyspark
from delta import *
from dags.lib.IncrementalLoader import IncrementalLoader
from dags.lib.Processer import *

In [3]:
def create_spark_session():
    conf = (
        pyspark.conf.SparkConf()
        .setAppName("LetsTalk")
        .set(
            "spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog",
        )
        .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .set("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
        .set("spark.hadoop.google.cloud.auth.service.account.enable", "true")
        .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/Users/alfio/projects/upc/BDMP2/gcs.json")
        .set("spark.sql.shuffle.partitions", "4")
        .set("spark.jars", "../jars/gcs-connector-hadoop3-latest.jar")
        .setMaster(
            "local[*]"
        )
    )

    builder = pyspark.sql.SparkSession.builder.appName("LetsTalk").config(conf=conf)
    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    return spark



In [4]:
spark = create_spark_session()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

:: loading settings :: url = jar:file:/Users/alfio/python_venv/general/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/alfio/.ivy2/cache
The jars for the packages stored in: /Users/alfio/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-82ff0f20-d09b-4d67-92ac-d08e990f2c78;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 68ms :: artifacts dl 2ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

In [5]:
loader = IncrementalLoader(spark, '/Users/alfio/projects/upc/BDMP2/data/letstalk_landing_zone_bdma', 'delta_news/sports')
df = loader.get_new_data()

25/05/02 17:22:40 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


CDF not available — doing full load


25/05/02 17:22:43 WARN DeltaHistoryManager: Found Delta commit 1 with a timestamp 1745921639653 which is greater than the next commit timestamp 1745921639219.
25/05/02 17:22:43 WARN DeltaHistoryManager: Found Delta commit 2 with a timestamp 1745921639654 which is greater than the next commit timestamp 1745921639603.
25/05/02 17:22:43 WARN DeltaHistoryManager: Found Delta commit 3 with a timestamp 1745921639655 which is greater than the next commit timestamp 1745921639603.
25/05/02 17:22:43 WARN DeltaHistoryManager: Found Delta commit 4 with a timestamp 1745921639656 which is greater than the next commit timestamp 1745921639653.
25/05/02 17:22:43 WARN DeltaHistoryManager: Found Delta commit 5 with a timestamp 1745921639657 which is greater than the next commit timestamp 1745921639445.
25/05/02 17:22:43 WARN DeltaHistoryManager: Found Delta commit 6 with a timestamp 1745921639658 which is greater than the next commit timestamp 1745921639603.
25/05/02 17:22:43 WARN DeltaHistoryManager: Fo

In [6]:
processor = NewsProcessor(spark, df)

In [7]:
processor.remove_clear_duplicates()
processor.name_to_id()
processor.remove_hidden_duplicates(['url'])

Removed 46 simple duplicate(s)
Removed 1 hidden duplicate(s)
